In [1]:
import pandas as pd
import numpy as np

### Model Training

In [2]:
df= pd.read_csv('data/gemstone.csv',index_col='id')
df.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
id,,,,,,,,,,
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


##### Independent and Dependent feature

In [3]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1]
X

,carat,cut,color,clarity,depth,table,x,y,z
id,,,,,,,,,
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77
...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62


In [4]:
#Segragating numerical and categorical columns
numerical_columns= X.select_dtypes(exclude='object').columns
categorical_columns=X.select_dtypes(include='object').columns

In [5]:
#Defining the custom ranking for each ordinal 
cut_categories= ['Fair','Good','Very Good','Premium','Ideal']
clarity_categories=['IF','VVS1','VVS2','VS1','VS2','SI1','SI2','I1']
color_categories=['D','E','F','G', 'H', 'I','J']

In [6]:
from sklearn.impute import SimpleImputer #Handling Missing Values
from sklearn.preprocessing import StandardScaler,OrdinalEncoder #Handling Categorical values and normalizing the data
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [7]:
#Numerical pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
    ]
)

#Categorical Pipeline
cat_pipeline=Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler' ,StandardScaler())
    ]
)

In [8]:
preprocessor=ColumnTransformer(
    transformers=[
        ('numerical_pipeline',num_pipeline,numerical_columns),
        ('categorical_pipeline',cat_pipeline,categorical_columns)
    ]
)

##### Train Test Split Data

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=42)

In [10]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [11]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor,AdaBoostRegressor

### Model Evaluation

In [17]:
models={
    'linear regression': LinearRegression(),
    'ridge':Ridge(),
    'lasso':Lasso(),
    'elasticnet':ElasticNet(),
    'knn':KNeighborsRegressor(),
    'gradientboosting':GradientBoostingRegressor(),
    'random forest':RandomForestRegressor(),
    'adaboost':AdaBoostRegressor()
}

In [18]:
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [19]:
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)
    trained_model_list.append(list(models.keys())[i])
    #predictions
    y_pred=model.predict(X_test)

    #Evaluating the model performance
    mae=mean_absolute_error(y_true=y_test,y_pred=y_pred)
    mse=mean_squared_error(y_true=y_test,y_pred=y_pred)
    rmse=np.sqrt(mse)
    r2=r2_score(y_true=y_test,y_pred=y_pred)
    r2_list.append(r2)
    print(f"Model: {type(model).__name__}")
    print(f"  Mean Absolute Error: {mae:.2f}")
    print(f"  Mean Squared Error: {mse:.2f}")
    print(f"  Root Mean Squared Error: {rmse:.2f}")
    print(f"  R-squared: {r2:.2f}")
    print("="*20)

Model: LinearRegression
  Mean Absolute Error: 675.08
  Mean Squared Error: 1029473.35
  Root Mean Squared Error: 1014.63
  R-squared: 0.94
Model: Ridge
  Mean Absolute Error: 675.11
  Mean Squared Error: 1029482.81
  Root Mean Squared Error: 1014.63
  R-squared: 0.94
Model: Lasso
  Mean Absolute Error: 676.24
  Mean Squared Error: 1029533.15
  Root Mean Squared Error: 1014.66
  R-squared: 0.94
Model: ElasticNet
  Mean Absolute Error: 1060.94
  Mean Squared Error: 2351174.87
  Root Mean Squared Error: 1533.35
  R-squared: 0.85
Model: KNeighborsRegressor
  Mean Absolute Error: 351.58
  Mean Squared Error: 454943.45
  Root Mean Squared Error: 674.49
  R-squared: 0.97
Model: GradientBoostingRegressor
  Mean Absolute Error: 332.35
  Mean Squared Error: 383434.88
  Root Mean Squared Error: 619.22
  R-squared: 0.98
Model: RandomForestRegressor
  Mean Absolute Error: 309.99
  Mean Squared Error: 373602.35
  Root Mean Squared Error: 611.23
  R-squared: 0.98
Model: AdaBoostRegressor
  Mean Abso